# Scattering Matrix

In this code, we implement the Scattering Matrix formalism to obtain the transmission and emission of multilayered systems.

## General imports

In [ ]:
# Libraries needed to import for this example

import numpy as np                    # Basic numerical python library
import scipy as sci                   # Algorithms for scientific computing
import matplotlib.pyplot as plt       # Plotting library for python

# Silica permittivity

real_epsilon_SiO2 = np.genfromtxt('Datasets/epsilon_real_SiO2.dat') # Real part of silica permittivity
imag_epsilon_SiO2 = np.genfromtxt('Datasets/epsilon_imag_SiO2.dat') # Imaginary part of silica permittivity

wSiO2_raw = real_epsilon_SiO2[:,0]/(6.58*10**(-16)*10**14) # Sampling frequencies for silica permittivity raw, 10^14 rad/s
eps_SiO2_raw = real_epsilon_SiO2[:,1]+1j*imag_epsilon_SiO2[:,1] # Full silica permittivity raw

wSiO2 = np.linspace(0.2,3-0.0001,200) # Sampling frequencies for silica permittivity, 10^14 rad/s
eps_SiO2 = np.interp(wSiO2,wSiO2_raw,eps_SiO2_raw) # Full silica permittivity

# Silver permittivity

Ag_raw = np.genfromtxt('Datasets/Ag_data.txt') # File containing all the Ag data

wAg_raw = 10**(-14)*(2*np.pi)*3*10**8/(Ag_raw[:,1]*10**(-6)); # Sampling frequencies for silver permittivity raw, 10^14 rad/s
eps_Ag_raw = Ag_raw[:,4] + 1j*Ag_raw[:,5] # Full silver permittivity raw

wAg = np.linspace(0.76,69,1500) # Sampling frequencies for silver permittivity, 10^14 rad/s
eps_Ag = np.interp(wAg,wAg_raw,eps_Ag_raw) # Full silver permittivity

# Silica permittivity for infrarred

SiO2_infrarred_wavelength_raw = np.genfromtxt('Datasets/Eps_SiO2_infrarred_wavelength.dat')
SiO2_infrarred_real_raw = np.genfromtxt('Datasets/Eps_SiO2_infrarred_real.dat')
SiO2_infrarred_imag_raw = np.genfromtxt('Datasets/Eps_SiO2_infrarred_imag.dat')

wSiO2_infrarred = 10**(-14)*(2*np.pi)*3*10**8/(SiO2_infrarred_wavelength_raw*10**(-6)) # Sampling frequencies for silica permittivity raw, 10^14 rad/s
epsSiO2_infrarred = np.interp(wAg,wSiO2_infrarred,SiO2_infrarred_real_raw+1j*SiO2_infrarred_imag_raw) # Full silica permittivity

# Permittivity of Drude metal

e_inf = 1 # Metal parameter e_inf
wp = 2.5 # Metal parameter wp (10**14 rad/s)
gamm = 0.01 # Metal para meter gamma (10**14 rad/s)

w_metal = wSiO2 # Sampling frequencies for metal permittivity, 10^14 rad/s

eps_metal = np.zeros((wSiO2.shape[0],)).astype('complex') # Will hold the metal permittivity values

for i in range(wSiO2.shape[0]):
    
    eps_metal[i] = e_inf - (wp**2)/(w_metal[i]*(w_metal[i]+1j*gamm))


## Unpatterned multilayered system: NFRHT

In [ ]:
# Defines the thermal derivative of the mean thermal energy of a mode of frequency wu

def thet(TT,wu):
    
    # Inputs
    
    # TT: temperature in SI
    # wu: frequency in SI
    
    # Parameters
    
    h = 6.626*10**(-34) # SI
    hbar = h/(2*np.pi) # SI
    kB = 1.381*10**(-23) # SI
    
    T_exp = np.exp((hbar*wu)/(kB*TT))
    
    values = (hbar**2*wu**2)/(4*np.pi**2*kB*TT**2)
    
    return (values*T_exp)/(T_exp-1)**2

In [ ]:
# Perform all calculations for the transmission of the system using Scattering Matrix

def k_funct(ku,NN,wu,epsss,ddd,gapd):
    
    # Inputs
    
    # ku: k value for the calculations
    # NN: number of material layers, the full vector has 2 extra components
    # wu: frequency for the calculations
    # epsss: material parameter vector, 2 components longer than NN
    # ddd: widths parameters, one extra 0 on each side
    # gapd: separation between the systems
    
    # Outputs
    
    # ktau: transmission times k value
    
    # Parameters
    
    c = 3*10**8 
    
    # Initialization
    
    fn = 0
    fn1 = 0
    D = 0
    
    I11 = np.zeros((2,2))
    I12 = np.zeros((2,2))
    I21 = np.zeros((2,2))
    I22 = np.zeros((2,2))
    
    S011 = np.eye(2)
    S012 = np.zeros((2,2))
    S021 = np.zeros((2,2))
    S022 = np.eye(2)
    
    S111 = np.zeros((2,2))
    S112 = np.zeros((2,2))
    S121 = np.zeros((2,2))
    S122 = np.zeros((2,2))
    
    wu = wu*10**14
    ku = ku*10**14
        
    # Main loop
        
    for n in range(NN+1):
        
        qn = np.sqrt(epsss[n]*wu**2-ku**2 + 0j)
        qn1 = np.sqrt(epsss[n+1]*wu**2-ku**2 + 0j)
        
        if np.imag(qn) < 0:
            
            qn = qn*(-1)
                
        if np.imag(qn+1) < 0:
            
            qn1 = qn1*(-1)
                
        D = qn/qn1
        
        fn = np.exp(1j*qn/c*ddd[n]*10**(-9))
        fn1 = np.exp(1j*qn1/c*ddd[n+1]*10**(-9))
        
        I11 = np.array([[D+1, 0.0],[0.0, epsss[n]/(D*epsss[n+1])+1]])/2
        I12 = np.array([[1-D, 0.0],[0.0, 1-epsss[n]/(D*epsss[n+1])]])/2
        I21 = I12
        I22 = I11
                
        S111 = fn*np.matmul(S011,np.linalg.inv(I11-fn*np.matmul(S012,I21)))
        S121 = np.matmul(S022,np.matmul(I21,S111))+S021
        S112 = np.matmul((fn*np.matmul(S012,I22)-I12)*fn1,np.linalg.inv(I11-fn*np.matmul(S012,I21)))
        S122 = np.matmul(S022,(np.matmul(I21,S112)+I22*fn1))
        
        S011 = S111
        S012 = S112
        S021 = S121
        S022 = S122
        
    rp = S121[1,1]*(-1)
    Rp = np.abs(S121[1,1])**2
    rs = S121[0,0]*(-1)
    Rs = np.abs(S121[0,0])**2
    
    qinter = np.sqrt(wu**2 - ku**2 + 0j)
    Ds = np.abs(1-rs*rs*np.exp(2*1j*qinter/c*gapd*10**(-9)))**2
    Dp = np.abs(1-rp*rp*np.exp(2*1j*qinter/c*gapd*10**(-9)))**2
    
    if ku < wu:
        
        taus = (1-Rs)**2/Ds
        taup = (1-Rp)**2/Dp
        
    else:
        
        taus = 4*np.imag(rs)**2*np.exp(-2*np.abs(qinter)/c*gapd*10**(-9))/Ds
        taup = 4*np.imag(rp)**2*np.exp(-2*np.abs(qinter)/c*gapd*10**(-9))/Dp
        
    ktau = (taus+taup)*ku
    
    return ktau

In [ ]:
# Consider the multilayer hyperbolic metamaterial

# Pre-allocations and constants

T = 300 # Temperature of the system, K

Nl = 8 # Number of layers to consider
Nw = wSiO2.shape[0] # Number of frequency points

epsd = 1 # Dielectric permittivty, we consider vacuum

c = 3*10**8 # Speed of light in SI

d0 = 10 # Width of the gap, nm
d_l = 10 # Width of each layer, nm. We assume all layers the same, we could change then individually if wanted

d = np.zeros((Nl+2)) # Width of all layers, including the infinite on above and below. THey have width 0.

for i in range(Nl):
    
    d[i+1] = d0
    
eps_layer = np.zeros((Nl+2,Nw)).astype('complex') # Permittivty of all layers

for i in range(Nl+2):
    
    for j in range(Nw):
        
        if np.mod(i,2) == 0:
            
            eps_layer[i,j] = epsd # Dielectric layer
            
        else:
            
            eps_layer[i,j] = eps_metal[j] # Metallic layer
            
spectral_HTC = np.zeros((Nw,)) # Will hold the HTC spectra
            
# Calculations

for i in range(Nw):
    
    eps_part = eps_layer[:,i] # Permittivties at current frequency
    
    part = sci.integrate.quad(lambda x: k_funct(x,Nl,wSiO2[i],eps_part,d,d0), 0, np.inf, limit = 10000, epsrel = 10**(-4)) # Integrate over k
    
    spectral_HTC[i] = thet(T,wSiO2[i]*10**14)*part[0]*10**14/c**2 
    
HTC = np.trapz(spectral_HTC,wSiO2*10**14) # HTC of this system

## Emissivty of a patterned system

In [ ]:
# We consider a patterned silica layer with a silver thin layer below
# We consider a square lattice of circular holes

# Pre-allocations and constants

c=3*10**8 # Speed of light, SI
h=6.626*10**(-34) # Planck's constant, SI
hbar = h/(2*np.pi)
kB=1.381*10**(-23) # Boltzmann constant, SI

Ntheta = 100 # Number of angles to consider
Nw = wAg.shape[0] # Number of frequency points to consider
Nl = 4 # Number of layers in the system, +2 infinite ones (above and below)

NGG = 5 # Number of G vectors to consider per value
NG = NGG**2 # Total number of G vectors to consider

dSiO2 = 500 # Width of the silica layer in microns
dAg = 0.12 # Width of the silver layer in microns

a = 0.3 # Lattice parameter in microns
R = 0.03 # Hole radius in microns
beta = np.pi*(R/a)**2 # Filling factor

theta = np.linspace(0,np.pi/2-0.0001,Ntheta) # Angles to consider for the calculation
d = np.array([0,dSiO2,dAg,0]) # All layer widths in the system

a0 = np.zeros((2*NG,1)).astype('complex'); # Field amplitudes
b0 = np.zeros((2*NG,1)).astype('complex'); 
aN = np.zeros((2*NG,1)).astype('complex');

eps1 = np.concatenate((np.reshape(eps_Ag,(Nw,1)),np.ones((Nw,1))),axis=1)
eps2 = np.concatenate((np.reshape(epsSiO2_infrarred,(Nw,1)),eps1),axis=1)
eps = np.concatenate((np.ones((Nw,1)),eps2),axis=1) # Permittivity of the ful system for all frequencies

G = np.zeros((2*NGG,NGG)) # G vectors matrix, each pair of columns holds the (x,y) elements of the vectors

emissivity = np.zeros((Nw,Ntheta)) # Will hold all the emissivity components

# Calculations

# Obtain all the G vectors (no units)

for i in range(NGG):
    
    for j in range(NGG):
        
        G[2*j,i] = j - np.floor(NGG/2)
        G[2*j+1,i] = i - np.floor(NGG/2)
        
G = G*(2*np.pi)/(a*10**(-6)) # Add units to the G vectors

for i in range(Nw): # Frequency loop
    
    k0 = wAg[i]/c*10**14 # Vacuum wavevector
    
    epse=np.zeros((NG,NG)).astype('complex') # Fourier of the permittivity
    
    # Obtain the Fourier transform of the permittivity, using the analytical solution
    
    for j in range(NG):
        
        for k in range(NG):
            
            GGGx = G[int(2*np.floor(j/NGG)),int(np.mod(j,NGG))] - G[int(2*np.floor(k/NGG)),int(np.mod(k,NGG))]
            GGGy = G[int(2*np.floor(j/NGG))+1,int(np.mod(j,NGG))] - G[int(2*np.floor(k/NGG))+1,int(np.mod(k,NGG))]
            
            GGG = np.sqrt(GGGx**2+GGGy**2)
            
            RR = R*10**(-6) # Radius with units
            
            if j == k:
                
                epse[j,k] = epsSiO2_infrarred[i] - beta*(epsSiO2_infrarred[i]-1)
                
            else:
                
                epse[j,k] = (4*epsSiO2_infrarred[i]*np.sin(GGGx*a)*np.sin(GGGy*a))/(a**2*(GGGx+GGGy*10**(-6))*(GGGy+GGGx*10**(-6))) - \
                (epsSiO2_infrarred[i]-1)/(a**2)*(0.5*np.pi*RR*sci.special.j1(GGG*RR)*sci.special.struve(0,GGG*RR)+
                                                0.5*sci.special.j0(GGG*RR)*(2*RR - np.pi*RR*sci.special.struve(1,GGG*RR))) 
                
    nu = np.linalg.inv(epse) # Inverse of permittivty, Fourier

    for th in range(Ntheta): # Angle loop
        
        kx=np.zeros((NG,NG)) # K matrices
        ky=np.zeros((NG,NG))
        
        Q = np.zeros((2*NG,2*NG)).astype('complex') # Q values
        
        f = np.zeros((2*NG,2*NG,Nl)).astype('complex') # f matrix
        
        M = np.zeros((4*NG,4*NG,Nl)).astype('complex') # Material matrices
        M1 = np.zeros((4*NG,4*NG,Nl)).astype('complex')
        
        S11 = np.zeros((2*NG,2*NG,Nl)).astype('complex') # Scattering matrices 
        S12 = np.zeros((2*NG,2*NG,Nl)).astype('complex')
        S21 = np.zeros((2*NG,2*NG,Nl)).astype('complex') 
        S22 = np.zeros((2*NG,2*NG,Nl)).astype('complex')
        
        # Obtain k matrices
    
        kxx=k0*np.sin(theta[th])
        kyy=k0*np.sin(theta[th])
        
        for j in range(NG):
                                
            kx[j,j]=kxx+G[int(2*np.floor(j/NGG)),int(np.mod(j,NGG))]
            ky[j,j]=kyy+G[int(2*np.floor(j/NGG))+1,int(np.mod(j,NGG))]
            
        # Solve the system to obtain the Q values
        
        W = k0**2*np.eye(2*NG)
        A2 = np.block([[nu, np.zeros((NG,NG)).astype('complex')], [np.zeros((NG,NG)).astype('complex'), nu]])
        A0 = (np.block([[np.matmul(ky,np.matmul(nu,ky)),-np.matmul(ky,np.matmul(nu,kx))],[-np.matmul(kx,np.matmul(nu,ky)),np.matmul(kx,np.matmul(nu,kx))]])
              +np.matmul(A2,np.block([[np.matmul(kx,kx),np.matmul(kx,ky)],[np.matmul(ky,kx),np.matmul(ky,ky)]]))-W)
        
        AAA,BBB = sci.linalg.eig(A0,-A2)
        
        for j in range(2*NG):
            
            val = np.sqrt(AAA[j])
            
            if np.imag(val) < 0:
                
                val = -val
                
            Q[j,j] = val
            
        # Obtain the f and M matrices
        
        for j in range(2*NG):
            
            for k in range(Nl):
                882
                f[j,j,k] = np.exp(1j*Q[j,j]*d[k]*10**(-6))
                
        for j in range(Nl):
            
            if j == 1:
                
                A0b = np.matmul(A2,np.block([[np.matmul(kx,kx),np.matmul(kx,ky)],[np.matmul(ky,kx),np.matmul(ky,ky)]]))
                G_not = np.matmul(A0b,np.matmul(BBB,np.linalg.inv(Q)))+np.matmul(A2,np.matmul(BBB,Q))
                M[:,:,j] = np.block([[G_not,-G_not],[BBB,BBB]])
                M1[:,:,j] = M[:,:,j]
                
            else:
                
                G1 = 1/(wAg[i]*10**14/c*np.sqrt(eps[i,j]-np.sin(theta[th])**2))*np.block([[(wAg[i]*10**14/c)**2*np.eye(NG),np.zeros((NG,NG)).astype('complex')],
                                                                                       [np.zeros((NG,NG)).astype('complex'),(wAg[i]*10**14/c*np.sqrt(eps[i,j]-np.sin(theta[th])**2))**2*np.eye(NG)]])
                M[:,:,j] = (wAg[i]*10**14/c)*np.block([[G1,-G1],[np.eye(2*NG),np.eye(2*NG)]])
                M1[:,:,j] = np.block([[np.linalg.inv(G1),np.eye(2*NG)],[-np.linalg.inv(G1),np.eye(2*NG)]])/(2*wAg[i]*10**14/c)
                
        # Obtain the scattering matrix
            
        for j in range(Nl-1):
                
            I = np.matmul(M1[:,:,j],M[:,:,j+1])
                
            index1row,index1col = np.indices([2*NG,2*NG])
            index2row,index2col = np.indices([2*NG,2*NG])
            index3row,index3col = np.indices([2*NG,2*NG])
            index4row,index4col = np.indices([2*NG,2*NG])
            
            index2col = index2col+2*NG
            index3row = index3row+2*NG
            index4row = index4row+2*NG
            index4col = index4col+2*NG
                
            I11 = I[index1row,index1col]
            I12 = I[index2row,index2col]
            I21 = I[index3row,index3col]
            I22 = I[index4row,index4col]
                
            S11[:,:,j+1]=np.matmul(np.linalg.inv(I11-np.matmul(f[:,:,j],np.matmul(S12[:,:,j],I21))),np.matmul(f[:,:,j],S11[:,:,j]));
            S12[:,:,j+1]=np.matmul(np.linalg.inv(I11-np.matmul(f[:,:,j],np.matmul(S12[:,:,j],I21))),np.matmul((np.matmul(f[:,:,j],np.matmul(S12[:,:,j],I22))-I12),f[:,:,j+1]));
            S21[:,:,j+1]=np.matmul(S22[:,:,j],np.matmul(I21,S11[:,:,j+1]))+S21[:,:,j];
            S22[:,:,j+1]=np.matmul(S22[:,:,j],np.matmul(I21,S12[:,:,j+1]))+np.matmul(S22[:,:,j],np.matmul(I22,f[:,:,j+1]));
            
            
        DN = np.sqrt(eps[i,Nl-1]-eps[i,0]*np.sin(theta[th]))/np.sqrt(eps[i,0]*np.cos(theta[th])**2)
        
        l_ind = int((NG+1)/2-1)
        v_ind = int((3*NG+1)/2-1)
        
        rot = np.array([[np.cos(theta[th]),0],[0,1]])
        
        # P-polarized components
        
        a0[v_ind] = 1.0+1j*0.0
        a0[l_ind] = -(M[l_ind,v_ind,0])/(M[l_ind,l_ind,0])*a0[v_ind]
        a00 = np.array([a0[l_ind],a0[v_ind]])
        a00 = np.matmul(rot,a00)
        a0[l_ind] = a00[0]
        a0[v_ind] = a00[1]
        
        b01 = np.matmul(S21[:,:,Nl-1],a0)
        aN1 = np.matmul(S11[:,:,Nl-1],a0)
        
        b0[l_ind] = b01[l_ind]
        b0[v_ind] = b01[v_ind]
        
        aN[l_ind] = aN1[l_ind]
        aN[v_ind] = aN1[v_ind]
        
        div = np.real(np.matmul(np.transpose(np.conj(a0)),np.matmul(np.conj(np.transpose(M[index3row,index3col,0])),np.matmul(M[index1row,index1col,0],a0))))
        rp = -np.matmul(np.real(np.matmul(np.transpose(np.conj(b0)),np.matmul(np.conj(np.transpose(M[index4row,index4col,0])),np.matmul(M[index2row,index2col,0],b0)))),np.linalg.inv(div))
        tp = np.matmul(np.real(np.matmul(np.transpose(np.conj(aN)),np.matmul(np.conj(np.transpose(M[index3row,index3col,0])),np.matmul(M[index1row,index1col,0],aN)))),np.linalg.inv(div))

        # S-polarized components
        
        a0[l_ind] = 1.0+1j*0.0
        a0[v_ind] = -(M[v_ind,l_ind,0])/(M[v_ind,v_ind,0])*a0[l_ind]
        a00 = np.array([a0[l_ind],a0[v_ind]])
        a00 = np.matmul(rot,a00)
        a0[l_ind] = a00[0]
        a0[v_ind] = a00[1]
        
        b01 = np.matmul(S21[:,:,Nl-1],a0)
        aN1 = np.matmul(S11[:,:,Nl-1],a0)
        
        b0[l_ind] = b01[l_ind]
        b0[v_ind] = b01[v_ind]
        
        aN[l_ind] = aN1[l_ind]
        aN[v_ind] = aN1[v_ind]
        
        div = np.real(np.matmul(np.transpose(np.conj(a0)),np.matmul(np.conj(np.transpose(M[index3row,index3col,0])),np.matmul(M[index1row,index1col,0],a0))))
        rs = -np.matmul(np.real(np.matmul(np.transpose(np.conj(b0)),np.matmul(np.conj(np.transpose(M[index4row,index4col,0])),np.matmul(M[index2row,index2col,0],b0)))),np.linalg.inv(div))
        ts = np.matmul(np.real(np.matmul(np.transpose(np.conj(aN)),np.matmul(np.conj(np.transpose(M[index3row,index3col,0])),np.matmul(M[index1row,index1col,0],aN)))),np.linalg.inv(div))

        # Obtain finally the emissivity
        
        emissivity[i,th] = (2-np.abs(rs)**2-np.abs(ts)**2-np.abs(rp)**2-np.abs(tp)**2)/2
    